In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../input/train.csv")
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")

In [ ]:
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

def extract_features(df):
    # 1. FamilySize の作成
    # 課題: SibSp + Parch + 1 (自分) を計算して FamilySize カラムを作成してください
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # 2. 敬称 (Title) の抽出
    # 定石コードを提示します。正規表現で名前から敬称を抜き出します。
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    # 3. 敬称の統合
    # 課題: 希少な敬称（Dr, Rev, Captなど）を 'Rare' に、
    # 'Mlle' 'Ms' を 'Miss' に、'Mme' を 'Mrs' に置換してください。
    # ヒント: df['Title'].replace(['Mlle','Ms'], 'Miss', inplace=True) などを使用
    df["Title"].replace(["Mlle", "Ms"], "Miss", inplace=True)
    df["Title"].replace("Mme", "Mrs", inplace=True)

    return df

train_data = extract_features(df)
test_data = extract_features(test)

# 結果の確認
print(train_data[['Title', 'Survived']].groupby(['Title']).mean())

In [ ]:
# 1. 敬称・客室クラスごとの年齢中央値を算出
# 課題: train_data を Title と Pclass でグループ化し、Age の中央値を算出してください。
# ヒント: df.groupby(['Title', 'Pclass'])['Age'].transform('median')

df.groupby(["Title", "Pclass"])["Age"].transform("median")


In [ ]:
# 2. 欠損値の穴埋め
train_data['Age'] = train_data['Age'].fillna(
    train_data.groupby(['Title', 'Pclass'])['Age'].transform('median')
)
test_data['Age'] = test_data['Age'].fillna(
    test_data.groupby(['Title', 'Pclass'])['Age'].transform('median')
)



In [ ]:
# 3. 年齢のビン化（子供フラグ）
# 補完後に、あなたが提案した「10歳以下か否か」のフラグを立てるのは有効な戦略です。
train_data['IsChild'] = (train_data['Age'] <= 10).astype(int)
test_data['IsChild'] = (test_data['Age'] <= 10).astype(int)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 1. 特徴量の選択（数値変数とカテゴリ変数）
# 既に Age 補完と IsChild, FamilySize の作成は終わっている前提です。
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Title", "FamilySize", "IsChild"]

# 2. カテゴリ変数の One-Hot Encoding
# 課題: pd.get_dummies を使い、train_data[features] を数値化してください。
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]

# 3. Random Forest の設定
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

# --- ここからが Expert の仕事 ---

# 4. K-Fold Cross Validation の実行
# 課題: cross_val_score を使い、5-fold CV で精度（accuracy）を計算してください。
# ヒント: scores = cross_val_score(model, X, y, cv=5)
scores = cross_val_score(model, X, y, cv=5)

print(f"CV Scores: {scores}")
print(f"Mean Accuracy: {scores.mean():.4f}")